# Schema

![](https://github.com/motional/nuplan-devkit/raw/master/docs/nuplan_schema.png)

## set environment variables

In [1]:
import os
NUPLAN_DATA_ROOT = os.getenv('NUPLAN_DATA_ROOT', './nuplan')
NUPLAN_MAPS_ROOT = os.getenv('NUPLAN_MAPS_ROOT', './nuplan/dataset/maps')
NUPLAN_DB_FILES = os.getenv('NUPLAN_DB_FILES', './nuplan/dataset/nuplan-v1.1/mini')
NUPLAN_MAP_VERSION = os.getenv('NUPLAN_MAP_VERSION', 'nuplan-maps-v1.0')

# ORM Database API

## load data

In [2]:
from nuplan.database.nuplan_db_orm.nuplandb_wrapper import NuPlanDBWrapper

nuplandb_wrapper = NuPlanDBWrapper(
    data_root=NUPLAN_DATA_ROOT,
    map_root=NUPLAN_MAPS_ROOT,
    db_files=NUPLAN_DB_FILES,
    map_version=NUPLAN_MAP_VERSION,
)

## query a database

In [3]:
log_db_name = "2021.05.12.22.00.38_veh-35_01008_01518"
log_db = nuplandb_wrapper.get_log_db(log_db_name)

Get all file name

In [4]:
from os import listdir
from os.path import isfile, join
file_names_with_extension = [f for f in listdir(NUPLAN_DB_FILES) if isfile(join(NUPLAN_DB_FILES, f))]

file_names = [name[:-3] for name in file_names_with_extension]
file_names

['2021.10.11.07.12.18_veh-50_00211_00304',
 '2021.08.17.17.17.01_veh-45_02314_02798',
 '2021.06.09.17.23.18_veh-38_00773_01140',
 '2021.10.11.02.57.41_veh-50_00352_00535',
 '2021.06.09.12.39.51_veh-26_05620_06003',
 '2021.08.17.18.54.02_veh-45_00665_01065',
 '2021.06.14.18.33.41_veh-35_03901_04264',
 '2021.05.12.23.36.44_veh-35_02035_02387',
 '2021.08.24.13.12.55_veh-45_00386_00472',
 '2021.10.11.08.31.07_veh-50_01750_01948',
 '2021.07.24.23.50.16_veh-17_01696_02071',
 '2021.10.01.19.16.42_veh-28_03307_03808',
 '2021.10.05.07.10.04_veh-52_01442_01802',
 '2021.07.16.20.45.29_veh-35_00600_01084',
 '2021.07.16.20.45.29_veh-35_01095_01486',
 '2021.10.06.07.26.10_veh-52_00006_00398',
 '2021.07.24.20.37.45_veh-17_00015_00375',
 '2021.06.08.12.54.54_veh-26_04262_04732',
 '2021.06.08.16.31.33_veh-38_01589_02072',
 '2021.07.16.18.06.21_veh-38_03231_03712',
 '2021.09.16.15.12.03_veh-42_01037_01434',
 '2021.06.23.20.43.31_veh-16_03607_04007',
 '2021.05.12.22.28.35_veh-35_00620_01164',
 '2021.06.2

## acess data

In [5]:
len(log_db.scenario_tag)

13812

In [6]:
len(log_db.lidar_pc)

10200

In [7]:
raw_scenario = [log_db.scenario_tag[i].type for i in range(len(log_db.scenario_tag))]
set(raw_scenario)

{'high_magnitude_speed',
 'low_magnitude_speed',
 'medium_magnitude_speed',
 'near_construction_zone_sign',
 'near_high_speed_vehicle',
 'near_pedestrian_on_crosswalk',
 'on_intersection',
 'on_stopline_traffic_light',
 'on_traffic_light_intersection',
 'starting_straight_stop_sign_intersection_traversal',
 'starting_straight_traffic_light_intersection_traversal',
 'stationary',
 'stationary_in_traffic',
 'traversing_crosswalk',
 'traversing_intersection',
 'traversing_traffic_light_intersection'}

In [8]:
log_db.lidar_box

LidarBox(1280922 entries):
--------------------------------------------------
token                       : f8745354d77f5e37
x                           : 664682.04248548
vy                          : -0.0020695771418821544
y                           : 3999281.5750495587
vz                          : -0.0018686033398442694
z                           : 606.1606478954054
yaw                         : -0.2767499453159221
lidar_pc_token              : e1e4ee25d1ff58f2
width                       : 0.40028481511936265
confidence                  : 0.5808950916575207
length                      : 0.397288774073045
track_token                 : 7af4e8cc4e86597a
height                      : 1.2509109225489499
next_token                  : b30624229d7d5a9a
vx                          : 0.008847108592404675
prev_token                  : None

token                       : 70153170580b51e5
x                           : 664689.4877757924
vy                          : -1.0953151492005082
y      

## filter

### import data type

In [9]:
from nuplan.database.nuplan_db_orm.lidar_pc import LidarPc
from nuplan.database.nuplan_db_orm.category import Category
from nuplan.database.nuplan_db_orm.ego_pose import EgoPose
from nuplan.database.nuplan_db_orm.lidar_box import LidarBox
from nuplan.database.nuplan_db_orm.log import Log
from nuplan.database.nuplan_db_orm.scene import Scene
from nuplan.database.nuplan_db_orm.scenario_tag import ScenarioTag
from nuplan.database.nuplan_db_orm.traffic_light_status import TrafficLightStatus
from nuplan.database.nuplan_db_orm.track import Track

### query and filter

In [10]:
scenario_tag_single = log_db.session.query(ScenarioTag) \
  .filter(ScenarioTag.agent_track_token != None) \
  .limit(10) \
  .all()

scenario_tag_single

[token                       : e83c784f4c005d26
 agent_track_token           : 6c253dd25e285c02
 lidar_pc_token              : 67fc50f5f2b354c8
 type                        : near_high_speed_vehicle
 ,
 token                       : 0bc0a62f8770519a
 agent_track_token           : 6c253dd25e285c02
 lidar_pc_token              : c99340a8526059ea
 type                        : near_high_speed_vehicle
 ,
 token                       : 5fc0e344b3b95773
 agent_track_token           : 6c253dd25e285c02
 lidar_pc_token              : 26a3b06a44f85761
 type                        : near_high_speed_vehicle
 ,
 token                       : 41fd65c8d65158bd
 agent_track_token           : 6c253dd25e285c02
 lidar_pc_token              : 83a8090c55045e8b
 type                        : near_high_speed_vehicle
 ,
 token                       : 93c86322bd0f5272
 agent_track_token           : 6c253dd25e285c02
 lidar_pc_token              : 04602bed0b8b5c1b
 type                        : near_high_speed_v

In [11]:
lidar_pc_token = scenario_tag_single[0].lidar_pc_token
agent_track_token = scenario_tag_single[0].agent_track_token

In [12]:
lidar_pc = log_db.session.query(LidarPc) \
  .filter(LidarPc.token == lidar_pc_token) \
  .all()
          
lidar_pc

[token                       : 67fc50f5f2b354c8
 prev_token                  : 4050dceb7cfb529b
 lidar_token                 : b95e61f2a75f5273
 scene_token                 : 964f7276202b566a
 timestamp                   : 1620857943850113
 next_token                  : c99340a8526059ea
 ego_pose_token              : bcb13eb64c205434
 filename                    : 2021.05.12.22.00.38_veh-35_01008_01518/MergedPointCloud/67fc50f5f2b354c8.pcd
 ]

In [13]:
prev_token = lidar_pc[0].prev_token
lidar_pc = log_db.session.query(LidarPc) \
  .filter(LidarPc.token == prev_token) \
  .all()
          
lidar_pc

[token                       : 4050dceb7cfb529b
 prev_token                  : 26d245d87c805769
 lidar_token                 : b95e61f2a75f5273
 scene_token                 : 964f7276202b566a
 timestamp                   : 1620857943800102
 next_token                  : 67fc50f5f2b354c8
 ego_pose_token              : 4fc68f21814a5474
 filename                    : 2021.05.12.22.00.38_veh-35_01008_01518/MergedPointCloud/4050dceb7cfb529b.pcd
 ]

# NuPlanScenarios

### import

In [14]:
import itertools
import logging
import random
from collections import defaultdict
from dataclasses import dataclass
from os.path import join
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import numpy.typing as npt

from nuplan.common.actor_state.vehicle_parameters import get_pacifica_parameters
from nuplan.common.maps.nuplan_map.map_factory import NuPlanMapFactory, get_maps_db
from nuplan.database.nuplan_db.nuplan_scenario_queries import (
    get_lidarpc_token_map_name_from_db,
    get_lidarpc_token_timestamp_from_db,
    get_lidarpc_tokens_with_scenario_tag_from_db,
)
from nuplan.planning.nuboard.base.data_class import NuBoardFile, SimulationScenarioKey
from nuplan.planning.nuboard.base.experiment_file_data import ExperimentFileData
from nuplan.planning.nuboard.base.simulation_tile import SimulationTile
from nuplan.planning.scenario_builder.abstract_scenario import AbstractScenario
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario import NuPlanScenario
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_filter_utils import discover_log_dbs
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import (
    DEFAULT_SCENARIO_NAME,
    ScenarioExtractionInfo,
)

from tutorials.utils.tutorial_utils import get_scenario_type_token_map, get_default_scenario_from_token

In [15]:
data_root=NUPLAN_DATA_ROOT
map_root=NUPLAN_MAPS_ROOT
db_files=NUPLAN_DB_FILES
map_version=NUPLAN_MAP_VERSION

### Using `NuPlanScenario` class.
Query a desired scenario via `NuPlanScenario` class, e.g., 'near_high_speed_vehicle'.

In [16]:
query_scenario = 'near_high_speed_vehicle'

log_db_files = discover_log_dbs(db_files)
scenario_type_token_map = get_scenario_type_token_map(log_db_files)

scenario_type_token_map[query_scenario]

[('nuplan/dataset/nuplan-v1.1/mini/2021.05.12.22.00.38_veh-35_01008_01518.db',
  '67fc50f5f2b354c8'),
 ('nuplan/dataset/nuplan-v1.1/mini/2021.05.12.22.00.38_veh-35_01008_01518.db',
  'c99340a8526059ea'),
 ('nuplan/dataset/nuplan-v1.1/mini/2021.05.12.22.00.38_veh-35_01008_01518.db',
  '26a3b06a44f85761'),
 ('nuplan/dataset/nuplan-v1.1/mini/2021.05.12.22.00.38_veh-35_01008_01518.db',
  '83a8090c55045e8b'),
 ('nuplan/dataset/nuplan-v1.1/mini/2021.05.12.22.00.38_veh-35_01008_01518.db',
  '04602bed0b8b5c1b'),
 ('nuplan/dataset/nuplan-v1.1/mini/2021.05.12.22.00.38_veh-35_01008_01518.db',
  'b627b48fb7a25dd8'),
 ('nuplan/dataset/nuplan-v1.1/mini/2021.05.12.22.00.38_veh-35_01008_01518.db',
  '55e093bd1d0d57b6'),
 ('nuplan/dataset/nuplan-v1.1/mini/2021.05.12.22.00.38_veh-35_01008_01518.db',
  '29d689eb758658e1'),
 ('nuplan/dataset/nuplan-v1.1/mini/2021.05.12.22.00.38_veh-35_01008_01518.db',
  'dad59f3ba54b57cd'),
 ('nuplan/dataset/nuplan-v1.1/mini/2021.05.12.22.00.38_veh-35_01008_01518.db',
  '

In [17]:
log_db_file, token = random.choice(scenario_type_token_map[query_scenario])
scenario = get_default_scenario_from_token(data_root, log_db_file, token, map_root, map_version)

API params descriptions in `abstract_scenario.py`

### duration of the scenario

In [18]:
scenario.duration_s.time_s

14.900031

### number of frame the scenario contains

In [19]:
scenario.get_number_of_iterations()

150

### frame interval

In [20]:
scenario.database_interval # s

0.1

### initial ego state

location

In [21]:
scenario.initial_ego_state.agent.center

StateSE2(x=664431.6314556043, y=3998070.1075999793, heading=-1.5885110181726294)

velocity

In [22]:
scenario.initial_ego_state.agent.velocity

x: 10.272650748678078, y: -0.1884853023961456

acceleration

In [23]:
scenario.initial_ego_state.dynamic_car_state.center_acceleration_2d

x: -0.35982236684295027, y: 0.03324020891153866

### ego future trajectory

In [24]:
ego_future_trajectory = scenario.get_ego_future_trajectory(
    iteration=0, # iteration within scenario 0 <= scenario_iteration < get_number_of_iterations
    time_horizon=10, # the desired horizon to the future, s
    num_samples=100 # (Optional) interpolate if necessary
)

ego_future_trajectory

<generator object get_sampled_ego_states_from_db at 0x7f0c7f5a0b30>

In [25]:
traj = list(ego_future_trajectory)
traj

center

In [26]:
traj[0].agent.center

StateSE2(x=664431.6158743284, y=3998069.0889348104, heading=-1.5887672642652146)

velocity

In [27]:
traj[0].agent.velocity

x: 10.258965614391316, y: -0.1937283589360892

**acceleration is NOT given in trajectory**

### objects within time window

In [28]:
tracked_objs_within_t = scenario.get_tracked_objects_within_time_window_at_iteration(
    iteration=0,
    past_time_horizon=5,
    future_time_horizon=0
)

tracked_objs_within_t

DetectionsTracks(tracked_objects=<nuplan.common.actor_state.tracked_objects.TrackedObjects object at 0x7f0c39109ac0>)

In [29]:
tracked_objs_within_t.tracked_objects.get_agents()

 ...]

location

In [30]:
tracked_objs_within_t.tracked_objects.get_agents()[0].center

StateSE2(x=664434.2529061043, y=3998058.578659939, heading=-1.5860568537986746)

velocity

In [31]:
tracked_objs_within_t.tracked_objects.get_agents()[0].velocity

x: -0.11796717934003409, y: -10.780458660886744

**acceleration is NOT given in tracking**